In [6]:
%load_ext autoreload
%autoreload 2

In [13]:
%run mainTNR.py

relTemp= 0.995
Iteration:  0  of  2000 , Trunc. Error: 2.00822e-06,0.000283112
Iteration:  100  of  2000 , Trunc. Error: 2.00822e-06,6.58429e-05
Iteration:  200  of  2000 , Trunc. Error: 2.00822e-06,4.71489e-05
Iteration:  300  of  2000 , Trunc. Error: 2.00822e-06,4.7101e-05
RGstep: 0 , Truncation Errors: 2.00822e-06, 4.7101e-05, 4.33853e-05, 0.000102462
Iteration:  0  of  2000 , Trunc. Error: 1.20523e-06,0.00369753
Iteration:  100  of  2000 , Trunc. Error: 1.20523e-06,7.01867e-05
Iteration:  200  of  2000 , Trunc. Error: 1.20523e-06,5.53399e-05
Iteration:  300  of  2000 , Trunc. Error: 1.20523e-06,5.53143e-05
RGstep: 1 , Truncation Errors: 1.20523e-06, 5.53143e-05, 8.85435e-05, 0.000119954
Iteration:  0  of  2000 , Trunc. Error: 1.39265e-06,0.00435041
Iteration:  100  of  2000 , Trunc. Error: 1.39265e-06,8.14519e-05
Iteration:  200  of  2000 , Trunc. Error: 1.39265e-06,6.88852e-05
Iteration:  300  of  2000 , Trunc. Error: 1.39265e-06,6.88755e-05
RGstep: 2 , Truncation Errors: 1.39265e

In [1]:
%run mainScInvTNR.py

Iteration:  0  of  2000 , Trunc. Error: 6.08743e-10,4.06838e-06
Iteration:  100  of  2000 , Trunc. Error: 6.08743e-10,2.94729e-06
Iteration:  200  of  2000 , Trunc. Error: 6.08743e-10,2.48122e-06
Iteration:  300  of  2000 , Trunc. Error: 6.08743e-10,2.1954e-06
Iteration:  400  of  2000 , Trunc. Error: 6.08743e-10,1.98306e-06
Iteration:  500  of  2000 , Trunc. Error: 6.08743e-10,1.81134e-06
Iteration:  600  of  2000 , Trunc. Error: 6.08743e-10,1.67127e-06
Iteration:  700  of  2000 , Trunc. Error: 6.08743e-10,1.55939e-06
Iteration:  800  of  2000 , Trunc. Error: 6.08743e-10,1.4727e-06
Iteration:  900  of  2000 , Trunc. Error: 6.08743e-10,1.40773e-06
Iteration:  1000  of  2000 , Trunc. Error: 6.08743e-10,1.36077e-06
Iteration:  1100  of  2000 , Trunc. Error: 6.08743e-10,1.32807e-06
Iteration:  1200  of  2000 , Trunc. Error: 6.08743e-10,1.30612e-06
Iteration:  1300  of  2000 , Trunc. Error: 6.08743e-10,1.29188e-06
Iteration:  1400  of  2000 , Trunc. Error: 6.08743e-10,1.2829e-06
RGstep: 0 

In [14]:
#Todo: diagonalize in python

import h5py
f = h5py.File('rT.h5', 'r')
a_group_key = list(f.keys())[0]
rT=np.array(f[a_group_key])

f = h5py.File('gT.h5', 'r')
a_group_key = list(f.keys())[0]
gT=np.array(f[a_group_key])

In [ ]:
#Todo: diagonalize in python

def logScaleSuper(psi):
    return np.einsum(psi.reshape(chir,chig,chir,chig),[1,3,5,7],rT,[1,21,8,2],gT,[3,22,2,4],rT,[5,23,4,6],gT,[7,24,6,8]).reshape((chir*chig)**2)

Atemp = LinearOperator(((chir*chig)**2,(chir*chig)**2), matvec = logScaleSuper)

print("eig")
w = scipy.sparse.linalg.eigsh(Atemp, k=12, M=None, sigma=None, which='SM', v0=None, ncv=None, maxiter=300, tol=1e-5, return_eigenvectors=False, Minv=None, OPinv=None, mode='normal')